In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
x=np.random.rand(10,1,28,28)
x.shape

(10, 1, 28, 28)

In [3]:
print(x[0].shape)
print(x[1].shape)

(1, 28, 28)
(1, 28, 28)


In [4]:
x[0,0] # x[0][0]

array([[0.17337944, 0.5435715 , 0.72041371, 0.49197208, 0.27158317,
        0.34497627, 0.39024831, 0.10208854, 0.65714313, 0.65996547,
        0.26506938, 0.58773969, 0.14626514, 0.81261514, 0.93640554,
        0.25208051, 0.75061201, 0.1961721 , 0.78108355, 0.61919414,
        0.23630783, 0.02593622, 0.23222018, 0.73955667, 0.90246132,
        0.27767238, 0.29120974, 0.39703277],
       [0.75247369, 0.22690718, 0.48385789, 0.8130666 , 0.08334229,
        0.97448649, 0.67054737, 0.49842137, 0.89680421, 0.4119112 ,
        0.77683903, 0.19654477, 0.20840946, 0.63966655, 0.81318235,
        0.30565143, 0.0328064 , 0.33509259, 0.13459276, 0.23054233,
        0.18752494, 0.22004838, 0.42396149, 0.72867702, 0.10096713,
        0.56847711, 0.65097329, 0.96030834],
       [0.99515852, 0.99937293, 0.34636545, 0.76311391, 0.53800303,
        0.06063172, 0.4029814 , 0.57073484, 0.18123965, 0.98864418,
        0.55373061, 0.1144092 , 0.04492856, 0.16129886, 0.9174708 ,
        0.29404252, 0.1371

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import sys,os
sys.path.append('/content/drive/MyDrive/연구실/밑바닥부터 시작하는 딥러닝/common')

In [7]:
from util import im2col

In [8]:
x1=np.random.rand(1,3,7,7)
col1=im2col(x1, 5, 5, stride=1, pad=0)
print(col1.shape)

(9, 75)


In [9]:
x2=np.random.rand(10,3,7,7)
col2=im2col(x2, 5, 5, stride=1, pad=0)
print(col2.shape)

(90, 75)


In [10]:
class Convolution:
    def __init__(self, W, b, stride=1, pad=0):
        self.W=W
        self.b=b
        self.stride=stride
        self.pad=pad

    def forward(self, x):
        FN, C, FH, FW=self.W.shape
        N,C,H,W=x.shape
        out_h=int(1+(H+2*self.pad-FH)/self.stride)
        out_w=int(1+(W+2*self.pad-FW)/self.stride)

        col=im2col(x,FH, FW, self.stride, self.pad)
        col_W=self.W.shape(FN, -1).T
        out=np.dot(col, col_W)+self.b

        out=out.reshape(N, out_h, out_w, -1).transpose(0,3,1,2)

        return out

In [11]:
class Pooling:
    def __init__(self, pool_h, pool_w, stride=1, pad=0):
        self.pool_h=pool_h
        self.pool_w=pool_w
        self.stride=stride
        self.pad=pad

    def forward(self,x):
        N,C,H,W=x.shape
        out_h=int(1+(H-self.pool_h)/self.stride)
        out_w=int(1+(W-self.pool_w)/self.stride)

        col=im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
        col=col.reshape(-1, self.pool_h*self.pool_w)

        out=np.max(col, axis=1)

        out=out.reshape(N, out_h,out_w, C).transpose(0,3,1,2)

        return out

In [13]:
from collections import OrderedDict
class SimpleConvNet:
    def __init__(self, input_dim=(1,28,28), conv_param={'filter_num':30, 'filter_size':5, 'pad':0, 'stride':1}, hidden_size=100, output_size=10, weight_init_std=0.01):
        filter_num=conv_param['filter_num']
        filter_size=conv_param['filter_size']
        filter_pad=conv_param['pad']
        filter_stride=conv_param['stride']
        input_size=input_dim[1]
        conv_output_size=(input_size-filter_size+2*filter_pad)/filter_stride+1
        pool_output_size=int(filter_num*(conv_output_size/2)*(conv_output_size/2))

        self.params={}
        self.params['W1']=weight_init_std*np.random.randn(filter_num, input_dim[0], filter_size, filter_size)
        self.params['b1']=np.zeros(filter_num)
        self.params['W2']=weight_init_std*np.random.randn(pool_output_size, hidden_size)
        self.params['b2']=np.zeros(hidden_size)
        self.params['W3']=weight_init_std*np.random.randn(hidden_size, output_size)
        self.params['b3']=np.zeros(output_size)

        self.layers=OrderedDict()
        self.layers['Conv1']=Convolution(self.params['W1'], self.params['b1'], conv_param['stride'], conv_parma['pad'])
        self.layers['Relu1']=Relu()
        self.layers['Pool1']=Pooling(pool_h=2, pool_w=2, stride=2)
        self.layers['Affine1']=Affine(self.params['W2'], self.params['b2'])
        self.layers['Relu2']=Relu()
        self.layers['Affine2']=Affine(self.params['W3'], self.params['b3'])    
        self.last_layer=SoftmaxWithLoss()

    def perdict(self, x):
        for layer in self.layers.values():
            x=layer.forward(x)
        return x

    def loss(self,x, t):
        y=self.predict(x)
        return self.last_layer.forward(y,t)

    def gradient(self, x, t):
        self.loss(x,t)

        dout=1
        dout=self.last_layer.backward(dout)

        layers=list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout=layer.backward(dout)

        grads={}
        grads['W1']=self.layers['Conv1'].dW
        grads['b1']=self.layers['Conv1'].db
        grads['W2']=self.layers['Affine1'].dW
        grads['b2']=self.layers['Affine1'].db
        grads['W3']=self.layers['Affine2'].dW
        grads['b3']=self.layers['Affine2'].db

        return grads